In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import time

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

In [7]:
url = "http://203.64.136.248/webpacIndex.jsp"
ISBN = "9789869109321"

# 初始化 driver；設定 explicit waits
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）

In [8]:
driver.get("http://192.192.231.232/webpacIndex.jsp")

In [11]:
driver.page_source

'<html xmlns="http://www.w3.org/1999/xhtml"><head>\n\n<script type="text/javascript" async="" src="http://www.google-analytics.com/ga.js"></script><script>\n\tif(document.location.toString().indexOf("#") != -1){\n\t\tlocation = "?" + document.location.toString().split("#")[1];\n\t}\n</script>\n \n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<meta name="Title" content="HyLib 整合性圖書館自動化系統">\n<meta name="Keywords" content="Webpac 2.0 Chinese Version">\n\n\n\n\n\n\n\n\n\n<link rel="stylesheet" type="text/css" href="/css/default.css">\n<link rel="stylesheet" type="text/css" href="/css/design.css">\n<link rel="stylesheet" type="text/css" href="/css/SpryTabbedPanels.css">\n<link rel="stylesheet" type="text/css" href="/css/thickbox.css" media="screen"> \n<link rel="stylesheet" type="text/css" media="all" href="/css/calendar-win2k-1.css">\n<link rel="stylesheet" type="text/css" media="all" href="/css/ui.datepicker.css">\n<link rel="stylesheet" type="text/css" href="/css/a

In [10]:
driver.find_element_by_id("totalpage").text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="totalpage"]"}
  (Session info: chrome=91.0.4472.124)


In [5]:
# 進入＂搜尋主頁＂
driver.get(url)

try:
    # 定位＂下拉式選單＂，選擇 ISBN
    search_field = wait.until(
        EC.presence_of_element_located((By.NAME, "search_field"))
    )
    select = Select(search_field)
    select.select_by_value("ISBN")

    # 抓取在哪間＂圖書館＂
    org = driver.title

    # 定位＂搜尋欄＂，輸入 ISBN
    search_input = wait.until(
        EC.presence_of_element_located((By.NAME, "search_input"))
    )
    search_input.send_keys(ISBN)
    search_input.submit()

    # 依據＂div.mainCon＂來判斷搜尋結果是＂一筆＂還是＂多筆＂
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.mainCon"))
        )

        # 定位＂表格＂，爬取內容
        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
        )
        tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]
        tgt["圖書館"] = org
        tgt["連結"] = driver.current_url
    except (NoSuchElementException, TimeoutException):
        if ".jsp" in url:  # 用來組合不同＂詳細書目＂的網址
            url = url.replace(url.split("/")[-1], "")

        tgt = []
        # 定位＂內嵌框架＂，切換至此內部，找尋多個＂詳細書目的網址＂
        iframe = wait.until(
            EC.presence_of_element_located((By.ID, "leftFrame"))
        )
        driver.switch_to.frame(iframe)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tgt_urls = []
        for anchor in soup.find_all("a", "bookname"):
            tgt_urls.append(url + anchor["href"])

        # 進入不同的＂詳細書目＂，再定位＂表格＂，爬取內容
        for urlx in urls:
            driver.get(urlx)
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
            )
            sec_tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]
            sec_tgt["連結"] = driver.current_url
            tgt.append(sec_tgt)
        tgt = pd.concat(tgt, axis=0, ignore_index=True)
        tgt["圖書館"] = org
    finally:
        driver.close()
        tgt.to_csv(f"{org}.csv", encoding="big5", errors="ignore")
        test_end = time.time()  # 測試用，計算程式進行時間
        print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")  # 測試用，計算程式進行時間
except Exception as e:
    print("="*80)
    print(f"程式執行失敗，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")
    print(f"網址為：「{test_url}」")
    print(f"錯誤訊息為：「{e}」")
    print("="*80)

NameError: name 'test_end' is not defined